In [55]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import openpyxl
from  openpyxl import *

In [54]:
def find_url_list():
    kitap = load_workbook("WebScraping.xlsm")
    url_list = []
    sheet=kitap['Links']
    for row in sheet.iter_rows(min_row=2, min_col=1, max_row=sheet.max_row, max_col=1):
        for cell in row:
            if cell.value is not None:
                url_list.append(cell.value)
    return url_list

In [53]:
def veri_cek(url): 
    page = requests.get(url)     #ilgili sayfayı python'a download edebilmek icin request modülünü kullanalım
    soup = BeautifulSoup(page.content, "html.parser")     # BeautifulSoup kütüphanesini kullanarak sitemizi bolumlerine ayıralım
    data = [] #cıktımızı elde etmek icin daha sonra kullanacagımız data listesi olusturalım
    
    uls = soup.find_all("ul") # Sayfadaki list elemanları arasında gezinebilmek icin unordered list'leri bulalım
    h3s = soup.find_all("h3") # Sayfadaki h3 dereceden basliklar arasında gezinebilmek icin h3 basliklari bulalım
    h4s = soup.find_all("h4") # Sayfadaki h4 dereceden basliklar arasında gezinebilmek icin h4 basliklari bulalım
    
    # Bazı sayfalarda hic h3 dereceden baslik olmayabiliyor, o yüzden hata fırlatmasına karşı try dongusune alalım
    # Biz sayfada General Characteristics, components ve performance başlığı altındaki listeleri istiyoruz. 
    # Ancak bu başlıklar bazı sayfalarda h3 bazı sayfalarda h4 olarak kodlanmış.
    
    try: 
        if any(soup.find(lambda tag:tag.name=="h3" and "characteristics" in tag.text)):
            
            current_baslik = h3s #Eger ilgili sayfadaki general characteristics basligi h3 derecedense o zaman h3 numaralı basliklar isimize yarayacaktır.
    except:
        current_baslik =  h4s # Eger ilgili sayfadaki general characteristics basligi h4 derecedense o zaman h4 numaralı basliklar isimize yarayacaktır.
            
            
    for header in current_baslik: #bir dongu yardımıyla ilgili baslik tipi (h3 veya h4) icinde gezinelim
        if (header.text == "General characteristics") or (header.text == "Components") or (header.text == "Performance"):
            ul = header.find_next("ul") #ilgili basliga geldik ve bir sonraki unordered listi elde etmek istiyoruz
            lis = ul.find_all("li") #ilgili unordered listedeki liste elemanlarını elde edelim 
    
            for li in lis: #elde ettigimiz liste elemanları arasında dongu dondurelim
                #Wikipedia sayfaları farklı sekilde HTML yapılarına sahip olabilir. Örn: bazı wikipedia sayfalarında "General characteristics" h3 baslik iken
                #bazı wikipedia sayfalarında h4 olarak kodlanmış. Aynı şekilde bazı wikipedia sayfalarında ilgili motor ismi Specifications başlığının yanında parantez
                #icinde yazarken, bazı sayfalarda ise ya h1 basligi olarak, ya da bir önceki h3 basligi olarak bulunmakta. Tüm wikipedia sayfaları ortak yapıda olmadığından
                #islem biraz daha karmasik hal almakta. 
                
                if "(" in li.find_previous("h2").text: #eger specifications basliginda motor ismi yazıyorsa
                    parantez_ici = li.find_previous("h2").text[li.find_previous("h2").text.find("(")+1:li.find_previous("h2").text.find(")")] #motor ismini elde ediyoruz
                    try:
                        if parantez_ici in li.find_previous("h3").text: #bazı sayfalarda parantez icinde yazan motor ismi olmayabiliyor, bu if dongusu bunu kontrol ediyor.
                            if li.find_previous("h3").find_previous("h2") != li.find_previous("h2"):
                                h3 = li.find_previous("h3")
                                h3_text = parantez_ici
                            else:
                                h3 = li.find_previous("h3")    
                                h3_text = h3.text.replace("[edit]", "") #Elde ettigimiz basliklar edit postfix'i ile gelmekte. İlgili kısmı siliyoruz.
                        else:
                            h3 = li.find_previous("h2")
                            h3_text = h3.text.replace("[edit]", "")
                            h3_text = h3_text.replace("[edit]","").split(" ",1)[1]
                            h3_text = re.sub(r'[()]', '', h3_text)
                    except:
                        #Daha önce açıkladığım gibi bazı sayfalarda h3 baslik bulunmamakta.
                        #Eger sayfada h3 basligi yoksa ve specificationun parantez icinde motor ismi yazıyorsa asagidaki donguye girer
                        
                        h3 = li.find_previous("h2")    
                        h3_text = h3.text.replace("[edit]", "") 
                        h3_text = h3_text.replace("[edit]","").split(" ",1)[1]
                        h3_text = re.sub(r'[()]', '', h3_text)
                else:
                        try:
                            h3 =  li.find_previous("h3")
                            # asagi kod prcagında eger specification ile listemiz arasında bir motor ismi yer alıyorsa onu aldırmasını saglıyoruz, 
                            # bazı durumlarda h3 basligi site'de olmasına ragmen motor ismi belirtmiyor.
                            if h3.find_previous("h2") != li.find_previous("h2"):
                                h3 = li.find_previous("h1")
                                h3_text = h3.text.replace("[edit]", "")
                            else:
                                h3_text = h3.text.replace("[edit]", "")
                        except:
                            h3 =  li.find_previous("h1")
                            h3_text = h3.text.replace("[edit]", "")
                        
                        
                h4 = li.find_previous("h4")
                try:
                    if not((h4.text == "General characteristics") or (h4.text == "Components") or (h4.text == "Performance")):
                        #print("buraya girdi")
                        h4 = li.find_previous("h3")
                except:
                    h4 = li.find_previous("h4")  
                
                if h3 and h4:
                    h4_text = h4.text.replace("[edit]", "")
                    if ":" in li.text: #Burada elde ettigimiz değerleri ":"'a göre ayırıyoruz.
                        #Bazı degerler birden fazla veri icerebilir. (Örn: maximum thrust genellikle birden fazla alt maddeden oluşuyor). Böyle olması durumunda
                        #Asagida bir if dongusu acarak ona gore islem yapmasını sağlıyorum. 
                        if "\n" in li.text.split(":",1)[1].strip(): 
                            for i in range(0,li.text.split(":",1)[1].strip().count("\n")+1):
                                data.append([h3_text, h4_text, li.text.split(":",1)[0].strip(), li.text.split(":",1)[1].strip().split("\n")[i].strip()])    
                        else: # Bazı verilerde ":" 
                            data.append([h3_text, h4_text, li.text.split(":",1)[0].strip(), li.text.split(":",1)[1].strip()])
                    #eger : yoksa asagidaki else'e girer
                    else:
                       data.append([h3_text, h4_text, "", li.text])     
                        
                        
                        
                else:
                    if h4==None:
                        h4 = li.find_previous("h3")
                    
                    h4_text = h4.text.replace("[edit]","")
                    #eger h3 baslik yoksa h2 basligin icindeki parantezi alalım,
                    try:
                        #Eger specifications basliginda parantez yoksa yani motor ismi belirtilmemişse ve h3/h4 baslik da sitede yoksa o zaman motor ismi olarak  
                        #h1 dereceden basligi atayalım diyorum.
                        h3_text = li.find_previous("h2").text.replace("[edit]","").split(" ",1)[1]
                        h3_text = re.sub(r'[()]', '', h3_text)
                    except:
                        h3_text = li.find_previous("h1").text
                    if ":" in li.text:
                        if "\n" in li.text.split(":",1)[1].strip():
                            for i in range(0,li.text.split(":",1)[1].strip().count("\n")+1):
                                data.append([h3_text, h4_text, li.text.split(":",1)[0].strip(), li.text.split(":",1)[1].strip().split("\n")[i].strip()])    
                        else:
                            data.append([h3_text, h4_text, li.text.split(":",1)[0].strip(), li.text.split(":",1)[1].strip()])
                    else:
                        data.append([h3_text, h4_text, "", li.text])     
    df = pd.DataFrame(data, columns=["Engine Type ", "Specification Name ", "Key", "Değer"]) #ilgili verilerden oluşan bir dataframe oluşturalım.
  
    return df


In [8]:
url_list = find_url_list()
df = pd.DataFrame(columns=["Engine Type ", "Specification Name ", "Key", "Değer"])
for i in range(len(url_list)):
    df = pd.concat([df,veri_cek(url_list[i])])
df

,Engine Type,Specification Name,Key,Değer
0,430D,General characteristics,Type,Direct drive two-stroke piston aircraft engine
1,430D,General characteristics,Bore,2.658 in (68 mm)
2,430D,General characteristics,Stroke,2.362 in (60 mm)
3,430D,General characteristics,Displacement,428 cc (26.11 cubic inches)
4,430D,General characteristics,Dry weight,65 lb (29 kg)
5,430D,General characteristics,Designer,McCornack/Cuyuna
6,430D,Components,Fuel type,Regular auto fuel
7,430D,Components,Oil system,40:1 premixed fuel and oil
8,430D,Components,Cooling system,fan forced air
9,430D,Performance,Power output,30 hp (22 kW)


In [7]:
import requests
from bs4 import BeautifulSoup

# Set the URL of the website
url = "https://en.wikipedia.org/wiki/General_Electric_T58"

# Send a GET request to the website
response = requests.get(url)

# Parse the HTML content of the website
soup = BeautifulSoup(response.text, "html.parser")

# Find the search box on the website
search_box = soup.find("input", {"name": "search"})

# Enter the value into the search box
search_box["value"] = "Hello World!"

# Submit the form
form = search_box.find_parent("form")
response = requests.post(form["action"], data={"search": search_box["value"]})


MissingSchema: Invalid URL '/w/index.php': No scheme supplied. Perhaps you meant http:///w/index.php?

In [10]:
import requests
from bs4 import BeautifulSoup

# Set the URL of the website
url = "https://en.wikipedia.org/wiki/Main_Page"

# Send a GET request to the website
response = requests.get(url)

# Parse the HTML content of the website
soup = BeautifulSoup(response.text, "html.parser")

# Find the search box on the website
search_box = soup.find("input", {"name": "search"})

# Enter the value into the search box
search_box["value"] = "General Electric T58"

# Reload the page with the search results
response = requests.get(url, params={"search": search_box["value"]})


In [14]:
import requests
from bs4 import BeautifulSoup

# Set the URL of the website
url = "https://en.wikipedia.org/wiki/Main_Page"

# Send a GET request to the website
response = requests.get(url)

# Parse the HTML content of the website
soup = BeautifulSoup(response.text, "html.parser")

# Find the search box on the website
search_box = soup.find("input", {"name": "search"})

# Enter the value into the search box
search_box["value"] = "General Electric T58"

# Reload the page with the search results
response = requests.get(url, params={"search": search_box["value"]})

# Parse the HTML content of the page with the search results
soup = BeautifulSoup(response.text, "html.parser")

# Get the URL of the page with the search results
search_url = response.url
search_url


'https://en.wikipedia.org/wiki/Main_Page?search=General+Electric+T58'

In [15]:
import requests
from bs4 import BeautifulSoup

# Set the URL of the website
url = "https://en.wikipedia.org/wiki/Main_Page"

# Send a GET request to the website
response = requests.get(url)

# Parse the HTML content of the website
soup = BeautifulSoup(response.text, "html.parser")

# Find the search box on the website
search_box = soup.find("input", {"name": "search"})

# Enter the value into the search box
search_box["value"] = "General Electric T58"

# Reload the page with the search results
response = requests.get(url, params={"search": search_box["value"]})

# Parse the HTML content of the page with the search results
soup = BeautifulSoup(response.text, "html.parser")

# Get the URL of the page with the search results
search_url = response.url

# Print the URL of the page with the search results
print(search_url)


https://en.wikipedia.org/wiki/Main_Page?search=Hello+World%21


In [17]:
import requests
from bs4 import BeautifulSoup

# Set the URL of the website
url = "https://en.wikipedia.org/wiki/Main_Page"

# Send a GET request to the website
response = requests.get(url)

# Parse the HTML content of the website
soup = BeautifulSoup(response.text, "html.parser")

# Find the search box on the website
search_box = soup.find("input", {"name": "search"})

# Enter the value into the search box
search_box["value"] = "General Electric T58"

# Reload the page with the search results
response = requests.get(url, params={"search": search_box["value"]})

# Parse the HTML content of the page with the search results
soup = BeautifulSoup(response.text, "html.parser")

# Get the URL of the page with the search results
search_url = response.url

# Print the URL of the page with the search results
print(search_url)


https://en.wikipedia.org/wiki/Main_Page?search=General+Electric+T58


In [19]:
import requests
from bs4 import BeautifulSoup

# Set the URL of the website
url = "https://en.wikipedia.org/wiki/Main_Page"

# Send a GET request to the website
response = requests.get(url)

# Parse the HTML content of the website
soup = BeautifulSoup(response.text, "html.parser")

# Find the search box on the website
search_box = soup.find("input", {"name": "search"})

# Enter the value into the search box
search_box["value"] = "General Electric T58"

# Reload the page with the search results
response = requests.get(url, params={"search": search_box["value"]})

# Check if the page was redirected
if response.history:
    # Get the URL of the last page in the redirect chain
    search_url = response.history[-1].url
else:
    # If there was no redirect, use the original URL
    search_url = response.url

# Print the URL of the page with the search results
print(search_url)


https://en.wikipedia.org/wiki/Main_Page?search=General+Electric+T58


In [27]:
import requests
from bs4 import BeautifulSoup

# Set the URL of the website
url = "https://en.wikipedia.org/wiki/Main_Page"

# Send a GET request to the website
response = requests.get(url)

# Parse the HTML content of the website
soup = BeautifulSoup(response.text, "html.parser")

# Find the search box on the website
search_box = soup.find("input", {"name": "search"})

# Enter the value into the search box
search_box["value"] = "General Electric T58"

# Reload the page with the search results
response = requests.get(url, params={"search": search_box["value"]})

reponse = response.url
# Check if the page was redirected
if response.history:
    # Follow the redirect chain to the final URL
    for resp in response.history:
        #response = requests.get(resp.headers["Location"])
        continue
    
    # Get the final URL
    search_url = response.url
else:
    # If there was no redirect, use the original URL
    search_url = response.url

# Print the URL of the page with the search results
print(search_url)
print(response.url)


https://en.wikipedia.org/wiki/Main_Page?search=General+Electric+T58
https://en.wikipedia.org/wiki/Main_Page?search=General+Electric+T58


In [21]:
import requests

# Set the URL of the website
url = "https://en.wikipedia.org/wiki/Main_Page"

# Send a GET request to the website
response = requests.get(url)

# Get the URL of the website
website_url = response.url

# Print the URL of the website
print(website_url)


https://en.wikipedia.org/wiki/Main_Page


In [28]:
import requests
from bs4 import BeautifulSoup

# Set the URL of the website
url = "https://en.wikipedia.org/wiki/Main_Page"

# Send a GET request to the website
response = requests.get(url)

# Parse the HTML content of the website
soup = BeautifulSoup(response.text, "html.parser")

# Find the search box on the website
search_box = soup.find("input", {"name": "search"})

# Enter the value into the search box
search_box["value"] = "Hello World!"

# Reload the page with the search results
response = requests.get(url, params={"search": search_box["value"]})

# Check if the page was redirected
if response.history:
    # Follow the redirect chain to the final URL
    for resp in response.history:
        response = requests.get(resp.headers["Location"])

    # Get the final URL
    search_url = response.url
else:
    # If there was no redirect, use the original URL
    search_url = response.url

# Print the URL of the page with the search results
print(search_url)


https://en.wikipedia.org/wiki/Main_Page?search=Hello+World%21


In [31]:
from bs4 import BeautifulSoup
import requests

# The search string
search_string = "General Electric F110"

# Make a request to the Wikipedia search page
page = requests.get('https://en.wikipedia.org/w/index.php?search=' + search_string)

# Parse the HTML using Beautiful Soup
soup = BeautifulSoup(page.content, 'html.parser')

# Extract the URL of the first Wikipedia page in the search results
page_url = soup.find('div', class_='mw-search-results').find('ul').find('li').find('a')['href']

# Print the URL
print(page_url)

AttributeError: 'NoneType' object has no attribute 'find'

In [36]:
from bs4 import BeautifulSoup
import requests

# The search string
search_string = "General Electric F110"

# Make a request to the Wikipedia search page
page = requests.get('https://en.wikipedia.org/w/index.php?search=' + search_string)

# Parse the HTML using Beautiful Soup
soup = BeautifulSoup(page.content, 'html.parser')

# Extract the URL of the first Wikipedia page in the search results
page_url = soup.find('div', class_='mw-search-results-wrap').find('ul').find('li').find('a')['href']

# Print the URL
print(page_url)


AttributeError: 'NoneType' object has no attribute 'find'

In [46]:
import urllib
import json

# The title of the Wikipedia page
title = "General Electric F110"

# Replace spaces in the title with %20
title_encoded = title.replace(' ', '%20')

# Make the request to the Wikipedia API
response = urllib.request.urlopen('https://en.wikipedia.org/w/api.php?action=query&format=json&titles=' + title_encoded + '&prop=info')

# Parse the response using the json module
data = json.loads(response.read())

# Print the JSON response to the console
print(data)

# Extract the URL from the response
page_id = list(data['query']['pages'].keys())[0]
page_url = data['query']['pages'][page_id]['fullurl']

# Print the URL
print(page_url)


{'batchcomplete': '', 'query': {'pages': {'636522': {'pageid': 636522, 'ns': 0, 'title': 'General Electric F110', 'contentmodel': 'wikitext', 'pagelanguage': 'en', 'pagelanguagehtmlcode': 'en', 'pagelanguagedir': 'ltr', 'touched': '2022-12-07T23:07:18Z', 'lastrevid': 1125036662, 'length': 20729}}}}


KeyError: 'fullurl'

In [48]:
from bs4 import BeautifulSoup
import requests

# The title of the Wikipedia page
title = "General Electric F110"

# Replace spaces in the title with _
title_encoded = title.replace(' ', '+')

# Make a request to the Wikipedia search page
page = requests.get('https://en.wikipedia.org/w/index.php?title=Special:Search&fulltext=1&search=' + title_encoded)

# Parse the HTML using Beautiful Soup
soup = BeautifulSoup(page.content, 'html.parser')

# Extract the URL of the first Wikipedia page in the search results
page_url = soup.find('div', class_='mw-search-results').find('ul').find('li').find('a')['href']

# Print the URL
print(page_url)


AttributeError: 'NoneType' object has no attribute 'find'

In [58]:
title = "General Electric F110"
title_new = title.replace(' ','+')
linkimiz = 'https://en.wikipedia.org/w/index.php?search=' + title_new
veri_cek(linkimiz)

,Engine Type,Specification Name,Key,Değer
0,F110-GE-129,General characteristics,Type,Afterburning turbofan
1,F110-GE-129,General characteristics,Length,181.9 in (462 cm)
2,F110-GE-129,General characteristics,Diameter,46.5 in (118 cm)
3,F110-GE-129,General characteristics,Dry weight,"3,920 lb (1,780 kg)"
4,F110-GE-129,Components,Compressor,"2 spool, 3-stage fan and 9-stage HP"
5,F110-GE-129,Components,Combustors,Annular
6,F110-GE-129,Components,Turbine,1-stage HP and 2-stage LP
7,F110-GE-129,Components,Bypass ratio,0.76:1
8,F110-GE-129,Performance,Maximum thrust,"Intermediate power: 17,155 lbf (76.31 kN)"
9,F110-GE-129,Performance,Maximum thrust,"Full afterburner: 29,400 lbf (131 kN)"
